In [1]:
import pyspark
from functools import reduce
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, Window, DataFrame
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import (sum as Fsum, min as Fmin, max as Fmax,
                                   col, when, isnull, mean,
                                   count, desc, asc, regexp_replace,
                                   udf, trim, lower, avg)
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.feature import (OneHotEncoder,
                                StringIndexer, VectorAssembler,
                                MinMaxScaler)
from pyspark.ml.classification import GBTClassifier, LogisticRegression
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import numpy as np

from functools import partial
from typing import Iterable

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1623898302873_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder.appName("Sparkify").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1623898302873_0006,pyspark,idle,Link,Link,✔


In [4]:
#spark.sparkContext.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
sc.install_pypi_package("pandas")
sc.install_pypi_package("matplotlib")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/51/51/48f3fc47c4e2144da2806dfb6629c4dd1fa3d5a143f9652b141e979a8ca9/pandas-1.2.4-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/24/33/5568d443ba438d95d4db635dd69958056f087e57e1026bee56f959d53f9d/matplotlib-3.4.2-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/8a/bb/488841f56197b13700afd5658fc279a2025a39e22449b7cf29864669b15d/pyparsing-2.4.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/33/34/542152297dcc6c47a9dcb0685eac6d652d878ed3cea83bf2b23cb988e857/Pillow-8.2.0-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e754c59dd546ffe1bbe732c8ab68b9c834e61/cycler-0.10.0-py2.py3-none-any.whl
  Using ca

In [6]:
import pandas as pd
import matplotlib.pyplot as plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
class SqlFeatureEngineer(Transformer):
    """
    Custom Transform to feature engineer using a SQL query
    """

    def __init__(self, table_name: str, agents: Iterable[str]):
        super(SqlFeatureEngineer, self).__init__()
        self.table_name = table_name
        
        sum_UA = "".join("\n        ,SUM(%s) as num_%s_interactions" % (agent, agent) for agent in agents)
        avg_UA = "".join("\n    ,AVG(sess.num_%s_interactions) as avg_%s_interactions" % (agent, agent) for agent in agents)

        session_query_select = """SELECT 
            sess.userId
            ,AVG(sess.num_items_in_session) as avg_num_items_in_session
            ,MAX(sess.longest_song_in_session) as longest_song
            ,AVG(sess.longest_song_in_session) as longest_song_per_session
            ,COUNT(sess.sessionId) as total_number_of_sessions
            ,SUM(sess.session_listening_time)/COUNT(sess.sessionId) as listening_time_per_session
            ,SUM(sess.number_of_songs)/COUNT(sess.sessionId) as avg_number_of_songs_per_session
            ,CASE
                WHEN SUM(sess.number_of_songs) <= 0 THEN 0
                ELSE SUM(sess.session_listening_time)/SUM(sess.number_of_songs)
            END as avg_song_length
            ,AVG(sess.session_length) as avg_session_length
            ,MAX(sess.session_end)-MIN(sess.registration) as time_since_joined
            ,MIN(sess.session_begin)-MIN(sess.registration) as time_to_first_session 
            ,CASE
                WHEN COUNT(sess.sessionId) <= 1 THEN 0
                ELSE ((MAX(sess.session_end)-MIN(sess.session_begin)) - SUM(sess.session_length))/(COUNT(sess.sessionId)-1)
            END as avg_time_between_sessions
            ,AVG(sess.gender_F) as avg_gender_F
            ,AVG(sess.gender_M) as avg_gender_M
            ,AVG(sess.gender_Unknown) as avg_gender_Unknown
            ,AVG(sess.status_200) as avg_status_200
            ,AVG(sess.status_307) as avg_status_307
            ,AVG(sess.status_404) as avg_status_404
            ,AVG(sess.num_paid_interactions) as avg_num_paid_interactions
            ,AVG(sess.num_free_interactions) as avg_num_free_interactions
            ,AVG(sess.num_put_interactions) as avg_num_put_interactions
            ,AVG(sess.num_get_interactions) as avg_num_get_interactions
            ,AVG(sess.num_nextsong_clicks) as avg_num_nextsong_clicks
            ,AVG(sess.num_thumbsup_clicks) as avg_num_thumbsup_clicks
            ,AVG(sess.num_home_visits) as avg_num_home_visits
            ,AVG(sess.num_addtoplaylist_clicks) as avg_num_addtoplaylist_clicks
            ,AVG(sess.num_addfriend_clicks) as avg_num_addfriend_clicks
            ,AVG(sess.num_rolladvert_visits) as avg_num_rolladvert_visits
            ,AVG(sess.num_logout_visits) as avg_num_logout_visits
            ,AVG(sess.num_thumbsdown_clicks) as avg_num_thumbsdown_clicks
            ,AVG(sess.num_downgrade_visits) as avg_num_downgrade_visits
            ,AVG(sess.num_settings_visits) as avg_num_settings_visits
            ,AVG(sess.num_help_visits) as avg_num_help_visits
            ,AVG(sess.num_upgrade_visits) as avg_num_upgrade_visits
            ,AVG(sess.num_about_visits) as avg_num_about_visits
            ,AVG(sess.num_savesettings_clicks) as avg_num_savesettings_clicks
            ,AVG(sess.num_errors) as avg_num_errors
            ,AVG(sess.num_upgrades) as avg_num_upgrades
            ,AVG(sess.num_downgrades) as avg_num_downgrades
            --,AVG(sess.num_cancel_visits) as avg_num_cancel_visits
            ,MAX(sess.cancelled) as cancelled"""

        session_query_from = """
        FROM (
            SELECT 
                userId
                ,sessionId
                ,MAX(itemInSession) as num_items_in_session
                ,CASE WHEN MAX(length) IS NULL THEN 0 ELSE MAX(length) END as longest_song_in_session
                ,CASE WHEN SUM(length) IS NULL THEN 0 ELSE SUM(length) END as session_listening_time
                ,CASE WHEN COUNT(song) IS NULL THEN 0 ELSE COUNT(song) END as number_of_songs
                ,min(ts) as session_begin
                ,max(ts) as session_end
                ,max(ts) - min(ts) as session_length
                ,CASE WHEN MIN(registration) IS NULL THEN MIN(ts) ELSE MIN(registration) END as registration
                ,MAX(cancelled) as cancelled
                ,MAX(gender_F) as gender_F
                ,MAX(gender_M) as gender_M
                ,MAX(gender_Unknown) as gender_Unknown
                ,SUM(status_200) as status_200
                ,SUM(status_307) as status_307
                ,SUM(status_404) as status_404
                ,SUM(level_paid) as num_paid_interactions
                ,SUM(level_free) as num_free_interactions
                ,SUM(method_PUT) as num_put_interactions
                ,SUM(method_GET) as num_get_interactions
                ,SUM(page_NextSong) as num_nextsong_clicks
                ,SUM(`page_Thumbs Up`) as num_thumbsup_clicks
                ,SUM(page_Home) as num_home_visits
                ,SUM(`page_Add to Playlist`) as num_addtoplaylist_clicks
                ,SUM(`page_Add Friend`) as num_addfriend_clicks
                ,SUM(`page_Roll Advert`) as num_rolladvert_visits
                ,SUM(page_Logout) as num_logout_visits
                ,SUM(`page_Thumbs Down`) as num_thumbsdown_clicks
                ,SUM(page_Downgrade) as num_downgrade_visits
                ,SUM(page_Settings) as num_settings_visits
                ,SUM(page_Help) as num_help_visits
                ,SUM(page_Upgrade) as num_upgrade_visits
                ,SUM(page_About) as num_about_visits
                ,SUM(`page_Save Settings`) as num_savesettings_clicks
                ,SUM(page_Error) as num_errors
                ,SUM(`page_Submit Upgrade`) as num_upgrades
                ,SUM(`page_Submit Downgrade`) as num_downgrades
                ,SUM(page_Cancel) as num_cancel_visits""" 

        session_query = session_query_select + avg_UA + session_query_from + sum_UA + """
            FROM """ + self.table_name + """
            GROUP BY
                userId,
                sessionId
        ) as sess
        GROUP BY
            userId
        """
        
        self.query = session_query

    def _transform(self, df: DataFrame) -> DataFrame:
        df.createOrReplaceTempView(self.table_name)
        summary = spark.sql(self.query)
        return summary

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# ### Step 2: Load and Clean Dataset

# Load the dataset from the AWS S3 bucket.
path = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"
user_log = spark.read.json(path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#user_log.select([count(when(col(c) == "", c).when(isnull(c), c)).alias(c) for c in user_log.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# There are missing values in userid and sessionid, so need to drop.
# Blank userIds are disguised as empty strings
user_log = user_log.dropna(how = "any", subset = ["userId", "sessionId"])
user_log = user_log.filter(user_log.userId != '')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#user_log.select([count(when(col(c) == "", c).when(isnull(c), c)).alias(c) for c in user_log.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Adjust the timestamp format from miliseconds to seconds for
# convenience
user_log = user_log.withColumn('ts', col('ts')/1000)
user_log = user_log.withColumn('registration', col('registration')/1000)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
user_log = user_log.drop("auth", "firstName", "lastName")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Filter rows with regex (find rows that match expression)
#user_log = user_log.withColumn('artist', regexp_replace('artist', r'[^0-9a-zA-Z]', ' '))
user_log = user_log.withColumn('userAgent', regexp_replace('userAgent', r'[^a-zA-Z]', ' '))
user_log = user_log.withColumn('userAgent', trim(lower(regexp_replace('userAgent', r'\s+', ' '))))
user_log = user_log.withColumn('userAgent', regexp_replace('userAgent', r'\s[a-z]\s', ' '))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
user_log = user_log.na.fill(value="Unknown",subset=["userAgent"])
user_log = user_log.na.fill(value="Unknown",subset=["gender"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#user_log.select([count(when(col(c) == "", c).when(isnull(c), c)).alias(c) for c in user_log.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# Keep a copy of the original column list for convenience
original_col_list = list(user_log.columns)

## One hot encode column variables

# Create string index for each column to be encoded
onehot_cols = ['gender', 'level', 'method', 'userAgent', 'page', 'status']
indexers = [
    StringIndexer(inputCol=column, outputCol=column+"_indexed")
    for column in onehot_cols
]

# Use the index column to prepare the one-hot encoder
encoders = [
    OneHotEncoder(
        inputCol=indexer.getOutputCol(),
        outputCol=indexer.getOutputCol()+"_encoded",
        dropLast=False
    ) 
    for indexer in indexers
]

# Generate a vector of encoded values using VectorAssembler for
# each row
assembler = [VectorAssembler(
    inputCols=[encoder.getOutputCol() for encoder in encoders],
    outputCol="onehot_features"
)]

# Run the string index, encoder, and assembler as a Pipeline,
# save the output back to the spark dataframe
onehot_pipeline = Pipeline(stages=indexers + encoders + assembler)
onehot_pipeline = onehot_pipeline.fit(user_log)
user_log = onehot_pipeline.transform(user_log)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# Generate the list of column names for convenience later (using
# VectorAssembler, the column names are not saved to the dataframe
# and need to be recovered)
#
# userAgent is renamed here because each value is a really long string
# and this makes the column name more convenient
expanded_oh_names = []
for i, column in enumerate(onehot_cols):
    for j, label in enumerate(onehot_pipeline.stages[i].labels):
        if column == 'userAgent':
            expanded_oh_names.append(column + '_' + str(j))
        else:
            expanded_oh_names.append(column + '_' + label)

# Generate list of columns to keep in the dataframe. For most purposes,
# this would be a list of the columns that were not encoded.
# If any of the columns in the list are one-hot encoded, this list
# will keep the original, non-encoded column in the dataframe
no_hot_cols = [c for c in original_col_list if c not in onehot_cols]

# create a new, expanded column list.
all_cols = no_hot_cols + expanded_oh_names

def extract(row, column_list):
    '''
    For each row, extract the one-hot values in the row vector (from
    VectorAssembler), along with any columns listed in column_list.
    
    Returns a tuple of values.
    '''
    col_tup = tuple(row[column] for column in column_list)
    return col_tup + tuple(row.onehot_features.toArray().tolist())

user_log = user_log.rdd.map(partial(extract, column_list=no_hot_cols)).toDF(all_cols)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# ### Step 3: Define Churn and Explore

user_log = user_log.withColumnRenamed('page_Cancellation Confirmation', 'cancelled')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# ### Step 4: Feature Engineering

# generate the arbitrary list of userAgent column names for passing
# to the SqlFeatureEngineer class
agents = [name for name in expanded_oh_names if "userAgent" in name]

table_name = 'user_log_temp'
user_summary = SqlFeatureEngineer(table_name, agents).transform(user_log)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# create a list of columns to min-max scale. Make sure to exclude
# the userId column and the target variable
columns_to_scale = [column for column in user_summary.columns if column != 'userId' and column != 'cancelled']

# min-max scaler
assembler = VectorAssembler(inputCols=columns_to_scale, outputCol="features")
transformed = assembler.transform(user_summary)
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel =  scaler.fit(transformed.select("features"))
user_scaled = scalerModel.transform(transformed)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
#user_summary.where(reduce(lambda x, y: x | y, (col(x).isNull() for x in user_summary.columns))).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
#user_log.filter(col('userId') == '1145574').filter(col('length').isNull()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
#user_summary.select([count(when(isnull(c), c)).alias(c) for c in user_summary.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# ### Step 5: Modeling

# drop userId
user_scaled = user_scaled.drop('userId')
user_log = []
user_summary = []

# Split into training and test data
(trainingData, testData) = user_scaled.randomSplit([0.75, 0.25], seed=42)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# Check base rate
#trainingData.select(mean(trainingData.cancelled)).show()
#testData.select(mean(testData.cancelled)).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Grid Search
logreg = LogisticRegression(labelCol="cancelled", featuresCol="scaledFeatures",
                            maxIter=100, regParam=0.01, elasticNetParam=0.5)
evaluator = MulticlassClassificationEvaluator(
  labelCol="cancelled", predictionCol="prediction",
  metricName="f1")

pipeline_lr = Pipeline(stages=[logreg])

paramGrid_lr = ParamGridBuilder().addGrid(logreg.elasticNetParam, [0, 0.33, 0.66, 1]).addGrid(logreg.regParam, [0.001, 0.01, 0.1]).build()
crossval_lr = CrossValidator(estimator=logreg,
                             estimatorParamMaps=paramGrid_lr,
                             evaluator=evaluator,
                             numFolds=3)

cvLRModel = crossval_lr.fit(trainingData)
bestLRModel = cvLRModel.bestModel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
predictions_lr = bestLRModel.transform(testData)

f1_lr = evaluator.evaluate(predictions_lr)
acc_lr = evaluator.setMetricName("accuracy").evaluate(predictions_lr)

print("LogReg Test Error = %g -- F1: %g" % (1.0 - acc_lr, f1_lr))

In [ ]:
coef = pd.DataFrame(list(bestLRModel.coefficients),
                    index=columns_to_scale,
                    columns=["Coefficients"])
coef['Abs. Value Coefficients'] = np.abs(coef['Coefficients'])
print("\n\n", coef.sort_values(['Abs. Value Coefficients'], ascending=False))

In [ ]:
sorted_coef = coef.sort_values(['Abs. Value Coefficients'], ascending=True)
sorted_coef['color'] = ['red' if x == -1 else 'blue' for x in np.sign(sorted_coef['Coefficients'])]
plt.barh(range(len(sorted_coef.index)), sorted_coef['Abs. Value Coefficients'],
         color=sorted_coef['color'])
plt.title('Coefficient Rankings')
plt.yticks(range(len(sorted_coef.index)), sorted_coef.index)